In [2]:
import os 
import pandas as pd
import json

parent_dir = os.path.dirname(os.getcwd())
json_path = os.path.join(parent_dir, 'data','raw','dataset_tripadvisor-reviews_2025-11-01_14-21-09-431.json')

with open(json_path, 'r', encoding="utf-8") as f:
    data = json.load(f) 

desired_keys = ['id', 'lang', 'rating', 'travelDate' ,'publishedDate', 'title', 'text', 
                'tripType']
filtered_data = []

for record in data:
    filtered_record = {key:record.get(key) for key in desired_keys if key in desired_keys}
    filtered_record['userLocation'] = ((record.get('user') or {}).get('userLocation') or {}).get('name')
    filtered_data.append(filtered_record)

df = pd.DataFrame(filtered_data)

In [3]:
df.head()

,id,lang,rating,travelDate,publishedDate,title,text,tripType,userLocation
0,1016346537,el,1,2025-07,2025-07-03,Απαράδεκτο grecotel.,Απαράδεκτο! Έφτασα μετά από πολύωρο ταξίδι στι...,BUSINESS,None
1,1015574543,en,5,2025-06,2025-06-28,Family vacation,I was afraid of what the accomodation would be...,FAMILY,None
2,987362026,el,3,2024-12,2025-01-02,Αδιαφορο,Το ξενοδοχείο χρήζει ανακαίνισης. Τα μαξιλάρια...,COUPLES,None
3,979445372,tr,1,2024-11,2024-11-10,Kahvaltı kuyruğu ve personelin kabalığı,Çocuklarla suit oda da kaldığımız bu otelde sa...,FAMILY,None
4,960799391,en,5,2023-08,2024-07-23,Place to stay in Alexandropoli,Wonderful stay. Rooms are clean and comfortabl...,COUPLES,None


In [4]:
df.shape

(19494, 9)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19494 entries, 0 to 19493
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             19494 non-null  object
 1   lang           19494 non-null  object
 2   rating         19494 non-null  int64 
 3   travelDate     19467 non-null  object
 4   publishedDate  19494 non-null  object
 5   title          19445 non-null  object
 6   text           19494 non-null  object
 7   tripType       19467 non-null  object
 8   userLocation   11260 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.3+ MB


*  <b>Convert data types</b> 
*  <b>Handle missing values</b>
*  <b>Remove duplicate values </b>
*  <b>Whitespace Cleaning</b>: Remove any extra spaces, tabs, or newlines to ensure a clean text format.
*  <b>URL Removal</b>: Reviews might contain URLs, which are usually not relevant for text analysis. You can use regular expressions to find and remove these. 
*  <b>Handle Emojis with Conversion</b>: Convert emojis into their textual descriptions.
*  <b>Translate non-english reviews</b>

In [6]:
df['travelDate'] = pd.to_datetime(df['travelDate'], format='%Y-%m' ,errors='coerce')
df['publishedDate'] = pd.to_datetime(df['publishedDate'], errors='coerce')

In [7]:
df.dtypes

id                       object
lang                     object
rating                    int64
travelDate       datetime64[ns]
publishedDate    datetime64[ns]
title                    object
text                     object
tripType                 object
userLocation             object
dtype: object

In [8]:
df.head()

,id,lang,rating,travelDate,publishedDate,title,text,tripType,userLocation
0,1016346537,el,1,2025-07-01,2025-07-03,Απαράδεκτο grecotel.,Απαράδεκτο! Έφτασα μετά από πολύωρο ταξίδι στι...,BUSINESS,None
1,1015574543,en,5,2025-06-01,2025-06-28,Family vacation,I was afraid of what the accomodation would be...,FAMILY,None
2,987362026,el,3,2024-12-01,2025-01-02,Αδιαφορο,Το ξενοδοχείο χρήζει ανακαίνισης. Τα μαξιλάρια...,COUPLES,None
3,979445372,tr,1,2024-11-01,2024-11-10,Kahvaltı kuyruğu ve personelin kabalığı,Çocuklarla suit oda da kaldığımız bu otelde sa...,FAMILY,None
4,960799391,en,5,2023-08-01,2024-07-23,Place to stay in Alexandropoli,Wonderful stay. Rooms are clean and comfortabl...,COUPLES,None


In [9]:
df['userLocation'].unique().tolist()

[None,
 'Istanbul, Türkiye',
 'Southwell, United Kingdom',
 'Cham, Switzerland',
 'Barcelona, Spain',
 'Hong Kong, China',
 'Brasov, Romania',
 'Hoddesdon, United Kingdom',
 'Petrosani, Romania',
 'Athens, Greece',
 'Seattle, Washington',
 'Edgware, United Kingdom',
 'Fordingbridge, United Kingdom',
 'Hamburg, Germany',
 'Hechtel-Eksel, Belgium',
 'Greater London, United Kingdom',
 'Fayetteville, North Carolina',
 'Jerusalem, Israel',
 'Paphos, Cyprus',
 'Berkshire, United Kingdom',
 'Liverpool, United Kingdom',
 'Manchester, United Kingdom',
 'Vienna, Austria',
 'Amsterdam, The Netherlands',
 'Renfrew, United Kingdom',
 'Ayrshire, United Kingdom',
 'Melton Mowbray, United Kingdom',
 'Zurich, Switzerland',
 'Chorley, United Kingdom',
 'Walthamstow, United Kingdom',
 'Hinckley, United Kingdom',
 'Pangbourne, United Kingdom',
 'Thornwood, New York',
 'Leeds, United Kingdom',
 'London, United Kingdom',
 'Homburg, Germany',
 'Glossa, Greece',
 'Paris, France',
 'Auckland Central, New Zeala

In [10]:
# Keep only the city from user location
df['userLocation'] = df['userLocation'].str.replace(r'^(.*?), (.*)$', r'\2', regex=True)

In [11]:
df['userLocation'].unique().tolist()

[None,
 'Türkiye',
 'United Kingdom',
 'Switzerland',
 'Spain',
 'China',
 'Romania',
 'Greece',
 'Washington',
 'Germany',
 'Belgium',
 'North Carolina',
 'Israel',
 'Cyprus',
 'Austria',
 'The Netherlands',
 'New York',
 'France',
 'New Zealand',
 'Belarus',
 'Bosnia and Herzegovina',
 'Michigan',
 'Australia',
 'Bulgaria',
 'Moldova',
 'Czech Republic',
 'Italy',
 'Mexico',
 'Sweden',
 'New Jersey',
 'Bahrain',
 'South Carolina',
 'Florida',
 'Hawaii',
 'Republic of North Macedonia',
 'United States',
 'Pennsylvania',
 'Connecticut',
 'Wisconsin',
 'Massachusetts',
 'Denmark',
 'Utah',
 'Costa Rica',
 'Texas',
 'South Africa',
 'California',
 'Finland',
 'Lithuania',
 'Maryland',
 'Minnesota',
 'United Arab Emirates',
 'Brunei Darussalam',
 'Serbia',
 'Saudi Arabia',
 'Reunion Island',
 'Canada',
 'Singapore',
 'Missouri',
 'New Mexico',
 'Norway',
 'Virginia',
 'Latvia',
 'Illinois',
 'Colombia',
 'Ukraine',
 'Poland',
 'DF',
 'District of Columbia',
 'Malta',
 'Ireland',
 'Ohio',


In [12]:
df['tripType'].unique()

array(['BUSINESS', 'FAMILY', 'COUPLES', 'NONE', 'FRIENDS', 'SOLO', None],
      dtype=object)

In [13]:
df['tripType'] = df['tripType'].replace('NONE', None)
df['tripType'].unique()

array(['BUSINESS', 'FAMILY', 'COUPLES', None, 'FRIENDS', 'SOLO'],
      dtype=object)

In [14]:
df[df['travelDate'].isna()]

,id,lang,rating,travelDate,publishedDate,title,text,tripType,userLocation
2116,8984,en,5,NaT,2024-07-15,Exceptional,We had the privilege of staying at the Grecote...,None,None
3764,5208400,en,2,NaT,2006-05-23,Lacking in customer service,"As with everything in Greece, the outward appe...",None,None
7359,3681709,en,4,NaT,2005-07-18,Good quality apartments,Flew out of Manchester with Thomsonfly (previo...,None,United Kingdom
11712,282522726,en,4,NaT,2015-06-23,Definitely a good choice,The hotel is located about 10 minutes from the...,None,China
14627,208459698,en,5,NaT,2014-06-02,Great hotel,This hotel is great in every way. Rooms are fr...,None,None
14633,200983085,en,5,NaT,2014-04-13,Excellent,Excellent hotel and Service! Beatyfull and cl...,None,None
15149,49686878,de,3,NaT,2008-12-26,Ausstattung gut f.Mittelklasse-Hotel,Zu wenig Essplätze bei regnerischem Wetter bzw...,None,None
15494,1679,fr,5,NaT,2024-07-15,Séjour détendu.,"Nous avons bien apprécié le cadre, la propreté...",None,None
15848,67875079,en,5,NaT,2010-06-18,Just perfect,Very close to everything. Excellent food. Very...,None,None
16675,49689652,de,4,NaT,2008-11-30,Top-Hotel,Wir hatten einen tollen Aufenthalt in Athen un...,None,None


In [15]:
df['text'].duplicated().any()

np.True_

In [16]:
df.loc[df.duplicated(subset=['text'], keep=False)]

,id,lang,rating,travelDate,publishedDate,title,text,tripType,userLocation
7568,915526761,fr,5,2023-09-01,2023-09-08,A paradise with a real family,J'ai passé les meilleures vacances de ma vie. ...,FAMILY,None
7569,915524989,fr,5,2023-09-01,2023-09-08,"Une maison, une famille",J'ai passé les meilleures vacances de ma vie. ...,FAMILY,None


In [17]:
df.drop_duplicates(subset=['text'], keep='first', inplace=True)
df['text'].duplicated().any()

np.False_

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19493 entries, 0 to 19493
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             19493 non-null  object        
 1   lang           19493 non-null  object        
 2   rating         19493 non-null  int64         
 3   travelDate     19466 non-null  datetime64[ns]
 4   publishedDate  19493 non-null  datetime64[ns]
 5   title          19444 non-null  object        
 6   text           19493 non-null  object        
 7   tripType       17379 non-null  object        
 8   userLocation   11260 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 1.5+ MB


In [19]:
def combine_review_text(row):
    title = row['title']
    text = row['text']
    
    has_title = pd.notna(title) and str(title).strip() != ""
    has_text = pd.notna(text) and str(text).strip() != ""
    
    if has_title and has_text:
        return str(title) + ". " + str(text)
    elif has_title:
        return str(title)
    elif has_text:
        return str(text)
    else:
        return None

In [20]:
# Merge review title and text in one column
df['review_text'] = df.apply(combine_review_text, axis=1)

# Drop the original 'title' and 'text' columns
df = df.drop(columns=['title', 'text'])

##### Whitespace cleaning

In [21]:
# Check for records where review title has leading or trailing spaces 
df[df['review_text'] != df['review_text'].str.strip()]['review_text'].tolist()

['Çok iyiydi 👍. Bütçeniz müsaitse paranıza kıyın ve kalın. Pişman olmayacaksınız. Biz ön bölümde alt katta denize ve havuza sıfır konumdaki odada kaldık. Havuz Eylülün 24 ünde bile çok iyiydi. Deniz de öyle. Odanızdan çıkıp birkaç adımda denizdesiniz. Havuz da zaten odanızın önünde. Güneşlenmek için herşey mevcut. Odaları çok iyi ve her şey var. Klima çok iyi soğutuyor. Banyo gayet geniş ve her türlü ihtiyacımız düşünülmüş.\nWifi deniz kenarında bile gayet hızlı ve iyi çalışıyor. Kahvaltı büfesi muhteşem ötesi diyebilirim. Yok yok yani o kadar. Pişi bile vardı. Havuz kenarında kahvaltı keyfi çok iyiydi. Spor salonunda her türlü spor aleti mevcut. Otelin konumu çok iyi. Restoran bölgelerine de yürüme mesafesinde. En güzeli de gün batımının odanızın önünden bile seyredilebilmesi. Biz bu oteli beğendik ',
 'Disappointment . As it is promoted It is a 5 star hotel and  they charge so. According to offer it is a 3 star my experience.  Also do not trust them  if the price for a room is given 

In [22]:
# Check for extra spaces, tabs or new lines
df[df['review_text'].str.contains(r'\s{2,}', regex=True)]['review_text'].to_list()

['Απαράδεκτο grecotel.. Απαράδεκτο! Έφτασα μετά από πολύωρο ταξίδι στις 15.30 και περίμενα μισή ώρα για να ελέγξουν αν είναι έτοιμο το δωμάτιο.  2 μέρες ήμουν χωρίς ίντερνετ που ήταν σημαντικό γιατί το ταξίδι ήταν επαγγελματικό.  Πουθενά wifi,  ουτε δεδομένα ακόμη και στο ipad με κάρτα ίντερνετ Sim, τίποτα.  Μου έφεραν ένα χαλασμένο φορητο ρουτερ και μου είπαν ότι καταλαβαίνουν.... αυτό χρειαζόμουν!!!!',
 'Αδιαφορο. Το ξενοδοχείο χρήζει ανακαίνισης. Τα μαξιλάρια υπερβολικά ψηλά με αποτελεσμα να έχεις άβολο ύπνο και τα στρώματα πολύ μαλακά. Το μπάνιο καλό. Το τραπεζάκι και οι καρέκλες στο μπαλκόνι σκουριάσμενες μηπως καποιος πρέπει να τις περιποιηθει????\n\nΤο πρωινό πλούσιο σε είδη αλλά όχι σε ποιότητα. Ο χυμός πορτοκαλιού δεν μπορουσε να τον πιεις με τιποτα!!Επιεικώς απαράδεκτος.\n\n Ο περιβάλλων χώρος χρήζει περιποιησης χαλάει την εικόνα του ξενοδοχείου.Καταθλιπτικος.\n\nΣε πολύ καλό σημείο της Αλεξανδρούπολη, καθαρό.Ευγενεια από το προσωπικο.\n\nΑκριβο γι αυτα που προσφερει. 3 Αστέρ

In [23]:
df['review_text'] = df['review_text'].str.replace(r'\s+', ' ', regex=True).str.strip()

In [24]:
df[df['review_text'] != df['review_text'].str.strip()]['review_text'].tolist()

[]

In [25]:
df[df['review_text'].str.contains(r'\s{2,}', regex=True)]['review_text'].to_list()

[]

In [26]:
df['review_text'] = df['review_text'].str.replace(r'\s+', ' ', regex=True).str.strip()

##### URL Removal

In [27]:
# Identify text containg urls
url_pattern = r'https?://\S+|www\.\S+'
df[df['review_text'].str.contains(url_pattern, regex=True)]['review_text'].tolist()

['В продолжение .... В продолжение ... Приближается лето и ... Мой первый отзыв об отеле тут: https://www.tripadvisor.ru/Hotel_Review-g189421-d680868-Reviews-Grecotel_Plaza_Spa_Apartments-Rethymnon_Rethymnon_Prefecture_Crete.html Собственно, считаю необходимым добавить\\изменить мнение об одном обстоятельстве. Ранее я писал: "Лежаки, зонтики на пляже бесплатно для проживающих и всегда в достатке." Так вот, в сезоне 2017 года наблюдались затруднения с наличием свободных лежаков. По моему личному мнению, затруднения связаны с ослаблением администрацией отеля контроля за соблюдением гостями правила - Не занимать лежаки заранее и освобождать лежаки ... Я наблюдал самый плохой вариант поведения отдыхающих - Занять лежаки у моря и одновременно у бассейна! Пр этом, Администрация отеля реагировала вяло, в отличие о прошлых лет. В остальном, предыдущий свой отзыв подтверждаю и едем в отель в этом сезоне. Всем рекомендую отель!',
 'Almost Like A Resort. Four of us stayed in Leoniki Residence sid

In [28]:
df[df['review_text'].str.contains(url_pattern, regex=True)]['review_text'].tolist()

['В продолжение .... В продолжение ... Приближается лето и ... Мой первый отзыв об отеле тут: https://www.tripadvisor.ru/Hotel_Review-g189421-d680868-Reviews-Grecotel_Plaza_Spa_Apartments-Rethymnon_Rethymnon_Prefecture_Crete.html Собственно, считаю необходимым добавить\\изменить мнение об одном обстоятельстве. Ранее я писал: "Лежаки, зонтики на пляже бесплатно для проживающих и всегда в достатке." Так вот, в сезоне 2017 года наблюдались затруднения с наличием свободных лежаков. По моему личному мнению, затруднения связаны с ослаблением администрацией отеля контроля за соблюдением гостями правила - Не занимать лежаки заранее и освобождать лежаки ... Я наблюдал самый плохой вариант поведения отдыхающих - Занять лежаки у моря и одновременно у бассейна! Пр этом, Администрация отеля реагировала вяло, в отличие о прошлых лет. В остальном, предыдущий свой отзыв подтверждаю и едем в отель в этом сезоне. Всем рекомендую отель!',
 'Almost Like A Resort. Four of us stayed in Leoniki Residence sid

In [29]:
# Remove urls from text data
df['review_text'] = df['review_text'].str.replace(url_pattern, '', regex=True)

df[df['review_text'].str.contains(url_pattern, regex=True)]

,id,lang,rating,travelDate,publishedDate,tripType,userLocation,review_text


##### Emoji conversion

In [30]:
import emoji 

def has_emoji(text:str):
    '''checks if a string contains any emojis'''
    return any(char in emoji.EMOJI_DATA for char in text) 

def demojize_text(text:str):
    '''checks if a text contains emoji and converts them to text'''

    # check if the string contains any emojis
    if any(char in emoji.EMOJI_DATA for char in text):
        return emoji.demojize(text, language='en')
    else:
        return text 


In [31]:
# Create a boolean Series indicating which rows have emojis
emoji_mask = df['review_text'].apply(has_emoji)

# Use the mask to display only the review texts that contain emojis
reviews_with_emojis = df.loc[emoji_mask, 'review_text']

In [32]:
reviews_with_emojis.to_list()

['Çok iyiydi 👍. Bütçeniz müsaitse paranıza kıyın ve kalın. Pişman olmayacaksınız. Biz ön bölümde alt katta denize ve havuza sıfır konumdaki odada kaldık. Havuz Eylülün 24 ünde bile çok iyiydi. Deniz de öyle. Odanızdan çıkıp birkaç adımda denizdesiniz. Havuz da zaten odanızın önünde. Güneşlenmek için herşey mevcut. Odaları çok iyi ve her şey var. Klima çok iyi soğutuyor. Banyo gayet geniş ve her türlü ihtiyacımız düşünülmüş. Wifi deniz kenarında bile gayet hızlı ve iyi çalışıyor. Kahvaltı büfesi muhteşem ötesi diyebilirim. Yok yok yani o kadar. Pişi bile vardı. Havuz kenarında kahvaltı keyfi çok iyiydi. Spor salonunda her türlü spor aleti mevcut. Otelin konumu çok iyi. Restoran bölgelerine de yürüme mesafesinde. En güzeli de gün batımının odanızın önünden bile seyredilebilmesi. Biz bu oteli beğendik',
 'Great place to stay. Excellent hotel, with very helpful and kind staff, beautiful surroundings, very clean rooms and tasty food. We liked it very much and definitely will come again 👍',


In [33]:
df['review_text'] = df['review_text'].apply(demojize_text)

In [34]:
# pattern to find common meta-characters used in text and emojis
meta_char_pattern = r'[\u200b-\u200d\ufe0f]'

df[df['review_text'].str.contains(meta_char_pattern, regex=True)]['review_text'].to_list()

["Exceptional Hospitality at Grecotel Corfu Imperial. \u200b\u200b\u200b\u200bExceptional Hospitality at Grecotel Corfu Imperial Hotel Experience Corfu Greco Hotel Imperial is a stunningly beautiful 5 star resort, which we visited in October 2024. The rooms were wonderful with some of the best hotel beds we have ever slept in. Every view from the windows was breathtaking and unspoilt. Breakfast and dinner buffets were plentiful varied and thoughtful.The food catered to everyone's individual needs and was delicious. Cleanliness throughout the hotel was perfect and gardens were beautifully maintained and thought about. Support for Neurodiverse Guests Most importantly for us, the hotel staff worked extremely hard to accommodate the needs of my autistic teenager. Without the level of genuine care and concern for my child's” safe food” needs, we would not have been able to enjoy our holiday, but all the staff we spoke to made enormous effort to source specific chips from across the island! 

In [35]:
df['review_text'] = df['review_text'].str.replace(meta_char_pattern, '', regex=True)

In [36]:
df.head()

,id,lang,rating,travelDate,publishedDate,tripType,userLocation,review_text
0,1016346537,el,1,2025-07-01,2025-07-03,BUSINESS,None,Απαράδεκτο grecotel.. Απαράδεκτο! Έφτασα μετά ...
1,1015574543,en,5,2025-06-01,2025-06-28,FAMILY,None,Family vacation. I was afraid of what the acco...
2,987362026,el,3,2024-12-01,2025-01-02,COUPLES,None,Αδιαφορο. Το ξενοδοχείο χρήζει ανακαίνισης. Τα...
3,979445372,tr,1,2024-11-01,2024-11-10,FAMILY,None,Kahvaltı kuyruğu ve personelin kabalığı. Çocuk...
4,960799391,en,5,2023-08-01,2024-07-23,COUPLES,None,Place to stay in Alexandropoli. Wonderful stay...


##### Reviews Translation

In [37]:

def calculate_translation_cost(dataframe):
    # Define DeepL api pro translation cost
    PRICE_PER_MILLION_CHARS = 20
    MILLION = 1_000_000
    
    reviews_to_translate = dataframe[
        (dataframe['lang'] != 'en') & 
        (dataframe['review_text'].notna())
    ]

    # Sum the character length of all reviews
    total_chars_to_translate = reviews_to_translate['review_text'].str.len().sum()
    
    # Calculate the cost
    estimated_cost = (total_chars_to_translate / MILLION) * PRICE_PER_MILLION_CHARS
    
    print(f"Total reviews in this dataset: {len(dataframe)}")
    print(f"Reviews to translate (non-English): {len(reviews_to_translate)}")
    print(f"Total characters to translate: {total_chars_to_translate:,.0f}")
    print(f"Estimated translation cost: €{estimated_cost:.2f}\n")

calculate_translation_cost(df)

Total reviews in this dataset: 19493
Reviews to translate (non-English): 7504
Total characters to translate: 6,348,111
Estimated translation cost: €126.96



In [ ]:
# TO_DO: TRANSLATE NON-ENGLISH REVIEWS

In [39]:
date_cols = ['travelDate', 'publishedDate'] 

for col in date_cols:
    if col in df.columns:
        # Convert the datetime objects to strings in 'YYYY-MM-DD' format.
        df[col] = df[col].dt.strftime('%Y-%m-%d')
        
        # Replace the 'NaT' string with None, so it becomes 'null' in the JSON.
        df[col] = df[col].replace({'NaT': None})

In [40]:
# Save cleaned data 
output_dir = os.path.join(parent_dir, 'data', 'cleaned')
os.makedirs(output_dir, exist_ok=True)
output_filepath = os.path.join(output_dir, 'cleaned_dataset_tripadvisor-reviews_2025-11-01_14-21-09-431.json')
df.to_json(output_filepath, orient='records', force_ascii=False, indent=2)

print(f"Successfully saved cleaned data to: {output_filepath}")


Successfully saved cleaned data to: c:\Users\Βασίλης\Desktop\Data Projects\Grecotel Project\data\cleaned\cleaned_dataset_tripadvisor-reviews_2025-11-01_14-21-09-431.json
